# Imports

In [2]:
import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go
# from ydata_profiling import ProfileReport
import numpy as np

dtype_mapping = {
    'propertyId': pd.StringDtype(),
    'localityName': 'category',
    'landMarks': pd.StringDtype(),
    'locality': pd.StringDtype(),
    'price': pd.Int64Dtype(),
    'nameOfSociety': pd.StringDtype(),
    'projectName': pd.StringDtype(),
    'carpetArea': pd.Int64Dtype(),
    'coveredArea': pd.Int64Dtype(),
    'carpetAreaSqft': pd.Int64Dtype(),
    'possessionStatus': pd.StringDtype(),
    'developerName': pd.StringDtype(),
    'flooringType': pd.StringDtype(),
    'floorNumber': pd.Int64Dtype(),
    'unitCountonFloor': pd.Int64Dtype(),
    'totalFloorNumber': pd.Int64Dtype(),
    'electricityStatus': pd.StringDtype(),
    'waterStatus': pd.StringDtype(),
    'longitude': pd.Float64Dtype(),
    'latitude': pd.Float64Dtype(),
    'transactionType': 'category',
    'facing': pd.StringDtype(),
    'ownershipType': pd.StringDtype(),
    'carParking': pd.StringDtype(),
    'furnished': 'category',
    'bedrooms': pd.Int64Dtype(),
    'bathrooms': pd.Int64Dtype(),
    'numberOfBalconied': pd.Int64Dtype(),
    'propertyType': 'category',
    'additionalRooms': pd.StringDtype(),
    'bookingAmountExact': pd.Int64Dtype(),
    'maintenanceChargesFrequency': 'category',
    'maintenanceCharges': pd.Int64Dtype(),
    'ageofcons': 'category',
    'isVerified': 'category',
    'listingTypeDesc': 'category',
    'premiumProperty': pd.BooleanDtype(),
    'noOfLifts': pd.Int64Dtype(),
    'propertyAmenities': pd.StringDtype(),
    'facilitiesDesc': pd.StringDtype(),
    'uuid': pd.StringDtype(),
    'flooringType_Vitrified': pd.BooleanDtype(),
    'flooringType_CeramicTiles': pd.BooleanDtype(),
    'flooringType_Marble': pd.BooleanDtype(),
    'flooringType_NormalTilesKotahStone': pd.BooleanDtype(),
    'flooringType_Granite': pd.BooleanDtype(),
    'flooringType_Wooden': pd.BooleanDtype(),
    'flooringType_Mosaic': pd.BooleanDtype(),
    'flooringType_Marbonite': pd.BooleanDtype(),
    'additionalRoom_PujaRoom': pd.BooleanDtype(),
    'additionalRoom_Study': pd.BooleanDtype(),
    'additionalRoom_Store': pd.BooleanDtype(),
    'additionalRoom_ServantRoom': pd.BooleanDtype(),
    'carParking_Open': pd.Int64Dtype(),
    'carParking_Covered': pd.Int64Dtype(),
    'ReservedParking': pd.BooleanDtype(),
}

COLUMNS_TO_DROP = [
    'coveredArea',
    'ReservedParking',
] + [
        'unitCountonFloor',
        'electricityStatus',
        'waterStatus',
        'facing',
        'bookingAmountExact',
        'isVerified',
        'listingTypeDesc',
        'maintenanceCharges',
        'maintenanceChargesFrequency',
        'latitude',
        'longitude',
        'carParking_Open',
        'carParking_Covered',
        'numberOfBalconied',
        'premiumProperty',
        'projectName',
        'nameOfSociety',
        'url',
        # 'uuid',
        'carpetAreaSqft',
        'noOfLifts',
        'ownershipType',
        'possessionStatus',
        'propertyType',

        'flooringType_Vitrified',
        'flooringType_CeramicTiles',
        'flooringType_Marble',
        'flooringType_NormalTilesKotahStone',
        'flooringType_Granite',
        'flooringType_Wooden',
        'flooringType_Mosaic',
        'flooringType_Marbonite',

        'additionalRoom_PujaRoom',
        'additionalRoom_Study',
        'additionalRoom_Store',
        'additionalRoom_ServantRoom',
        
        'landMarks', 
        'locality', 
        'developerName',]

################################################################################
# ONLY USING THE RAW SETs, NOT IMPUTED SET
################################################################################
df_train = pd.read_csv(
    'Data/train.csv',
    dtype = dtype_mapping,
    index_col=0
)
df_train.drop(COLUMNS_TO_DROP, axis=1, inplace=True)

df_test = pd.read_csv(
    'Data/test.csv',
    dtype = dtype_mapping,
    index_col=0
)
df_test.drop(COLUMNS_TO_DROP, axis=1, inplace=True)

################################################################################
# DROPPING ALL ROWS WITH MISSING VALUES
################################################################################

print("Train Set Null values: ", df_train.isna().sum(), '\n')
print("Test Set Null values: ", df_test.isna().sum(), '\n')

df_train.dropna(axis=0, inplace=True)
df_test.dropna(axis=0, inplace=True)

Train Set Null values:  localityName           0
price                  0
carpetArea          3764
floorNumber            0
totalFloorNumber       0
transactionType        0
furnished             37
bedrooms               0
bathrooms              0
ageofcons           2571
dtype: int64 

Test Set Null values:  localityName          0
price                 0
carpetArea          977
floorNumber           0
totalFloorNumber      0
transactionType       0
furnished            14
bedrooms              0
bathrooms             0
ageofcons           693
dtype: int64 



# Feature Encoding

In [3]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

# Assume that df_train and df_test are your already cleaned and imputed datasets.
X_train = df_train.drop("price", axis=1)
y_train = df_train["price"]

# List of numeric features
numeric_cols = [
    "carpetArea",
    "floorNumber",
    "totalFloorNumber",
    "bedrooms",
    "bathrooms",
]

# For the two features that will be encoded differently:
cat_diff_cols = ["localityName", "transactionType"]

# The remaining categorical columns that are on an inherently ordinal scale.
# Note: even though localityName might appear here, you may choose a different
# encoding strategy depending on its cardinality. In this code, we are treating
# it specially in the one-hot transformation.
ordinal_cols = ["furnished", "ageofcons"]


# You can now proceed to train your models using X_train_linear / X_train_tree and 
# evaluate using X_test_linear / X_test_tree.


## Linear

In [4]:
# =============================================================================
# Pipeline for linear models
#   - Numerical features: standard scaled.
#   - For transactionType, furnished: one-hot encoded.
#   - For ordinal_cols (localityName, ageofcons): ordinal-encoded
#     and then scaled (so that all features are on a similar scale).
# =============================================================================

# Define transformers
numeric_transformer = StandardScaler()

onehot_transformer = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

# Here we create a pipeline that first ordinal-encodes then scales the result.
furnished_order = ['Unfurnished', 'Semi-Furnished', 'Furnished']
ordinal_transformer_furnished = Pipeline(
    steps=[
        ("ordinal", OrdinalEncoder(categories=[furnished_order])),
        ("scaler", StandardScaler()),
    ]
)

# # Here we create a pipeline that first ordinal-encodes then scales the result.
# ordinal_transformer_rs = Pipeline(
#     steps=[
#         ("ordinal", OrdinalEncoder()),
#         ("scaler", StandardScaler()),
#     ]
# )

# Here we create a pipeline that first ordinal-encodes then scales the result.
age_order = [
    'Under Construction',  # first: youngest / newest state
    'New Construction',
    'Less than 5 years',
    '5 to 10 years',
    '10 to 15 years',
    '15 to 20 years',
    'Above 20 years'       # last: oldest
]
ordinal_transformer_ageofcons = Pipeline(
    steps=[
        ("ordinal", OrdinalEncoder(categories=[age_order])),
        ("scaler", StandardScaler()),
    ]
)

# Create the ColumnTransformer for the linear pipeline.
lin_preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("onehot", onehot_transformer, cat_diff_cols),
        ("ord-furnished", ordinal_transformer_furnished, ["furnished"]),
        # ("ord-reservedparking", ordinal_transformer_rs, ["ReservedParking"]),
        ("ord-ageofcons", ordinal_transformer_ageofcons, ["ageofcons"]),
    ]
)

# The overall pipeline (here you could add a linear model as the final estimator)
lin_pipeline = Pipeline(steps=[("preprocessor", lin_preprocessor)])

# Now transform the training features for the linear model:
X_train_linear = lin_pipeline.fit_transform(X_train)

## Tree (numeric only)

In [5]:
# =============================================================================
# Pipeline for tree-based models
#   - Numerical features: standard scaled.
#   - For all categorical features (transactionType, furnished, ReservedParking,
#     localityName, ageofcons): ordinal-encoded and then scaled.
# =============================================================================

# Create a pipeline for encoding the categorical features as ordinal then scaling.
tree_cat_transformer = Pipeline(
    steps=[
        ("ordinal", OrdinalEncoder()),
        ("scaler", StandardScaler()),
    ]
)

tree_preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("ord", tree_cat_transformer, cat_diff_cols),
        ("ord-furnished", ordinal_transformer_furnished, ["furnished"]),
        # ("ord-reservedparking", ordinal_transformer_rs, ["ReservedParking"]),
        ("ord-ageofcons", ordinal_transformer_ageofcons, ["ageofcons"]),
    ]
)

tree_pipeline = Pipeline(steps=[("preprocessor", tree_preprocessor)])
# Now transform the training features for the tree models:

X_train_tree = tree_pipeline.fit_transform(X_train)

## Tree (numeric and categorical)

In [6]:
# =============================================================================
# Pipeline for models which handle categorical features:
#   - Numerical features: standard scaled.
#   - For categorical features (furnished, ReservedParking,
#     , ageofcons): ordinal-encoded and then scaled.
#   - For categorical features (transactionType, localityName): Kept as is.
# =============================================================================

# Create a pipeline for encoding the categorical features as ordinal then scaling.

tree_preprocessor_gb = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("passthrough", "passthrough", cat_diff_cols),
        ("ord-furnished", ordinal_transformer_furnished, ["furnished"]),
        # ("ord-reservedparking", ordinal_transformer_rs, ["ReservedParking"]),
        ("ord-ageofcons", ordinal_transformer_ageofcons, ["ageofcons"]),
    ]
)

tree_pipeline_gb = Pipeline(steps=[("preprocessor", tree_preprocessor_gb)])
# Now transform the training features for the tree models:
X_train_gb = tree_pipeline_gb.fit_transform(X_train)

In [7]:
print("Linear model feature shape: train:", X_train_linear.shape)
print("Tree model feature shape: train:", X_train_tree.shape)
print("Tree model for GB feature shape: train:", X_train_gb.shape)

Linear model feature shape: train: (13983, 233)
Tree model feature shape: train: (13983, 9)
Tree model for GB feature shape: train: (13983, 9)


In [13]:
import pickle

with open("PipelinesAndModels/final_preprocessor_pipeline_iteration_3.pkl", "wb") as f:
    pickle.dump(tree_pipeline_gb, f)

# Training LightGBM model with best parameters

In [14]:
import numpy as np
import pandas as pd
import warnings

import lightgbm as lgb

# Ignore warnings for cleaner output
warnings.filterwarnings("ignore")

In [15]:
lgbm_reg = lgb.LGBMRegressor(
    random_state=42,
    learning_rate=0.1,
    n_estimators=500,
    max_depth=-1,
)
mapping = {
    "carpetArea": pd.Float64Dtype(),
    "floorNumber": pd.Float64Dtype(),
    "totalFloorNumber": pd.Float64Dtype(),
    "bedrooms": pd.Float64Dtype(),
    "bathrooms": pd.Float64Dtype(),
    "localityName": 'category',
    "transactionType": 'category',
    "furnished": pd.Float64Dtype(),
    "ageofcons": pd.Float64Dtype(),
}
X_train_gb_df = pd.DataFrame(X_train_gb, columns = numeric_cols + cat_diff_cols + ordinal_cols).astype(mapping)
y_train_df = pd.Series(y_train)

lgbm_reg.fit(X_train_gb_df, y_train_df, categorical_feature = ['localityName', 'transactionType'])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000327 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 588
[LightGBM] [Info] Number of data points in the train set: 13983, number of used features: 9
[LightGBM] [Info] Start training from score 11030358.469284


LGBMRegressor(n_estimators=500, random_state=42)

In [16]:
lgbm_reg

LGBMRegressor(n_estimators=500, random_state=42)

In [17]:
import pickle

with open("PipelinesAndModels/lgb_model_iteration_3.pkl", "wb") as f:
    pickle.dump(lgbm_reg, f)

# Catrgorical Options fro frontend

In [18]:
options = {}

In [19]:
options['localityName'] = sorted(list(df_train['localityName'].unique()))
options['transactionType'] = sorted(list(df_train['transactionType'].unique()))
options['ageofcons'] = age_order
options['furnished'] = furnished_order

In [20]:
import json
with open("PipelinesAndModels/options_iteration_3.json", "w") as f:
    json.dump(options, f)